# Agrupar glosas de diagnosticos similares

- En este notebook se quiere agrupar la glosa de diagnosticos luego de haberlos preprocesados
(segun la funcion que esta en src/data/build_model.py). Esto, ya que existe redundancia en las glosas
(Por ejemplo: 4 glosas distintas para la Tetralogia de Fallot).

- Para esto se utilizara K-Means, y se iterara en la cantidad de clusters a generar. Esta informacion
se podria obtener del equipo de Patologias Congenitas.

In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [5]:
COLS_A_PREPROCESAR_TEXTO = os.environ.get("COLS_A_PREPROCESAR_TEXTO").split(",")
COLS_INFO_SENSIBLE = os.environ.get("COLS_INFO_SENSIBLE").split(",")


In [16]:
import pandas as pd

from sklearn.cluster import KMeans
from src.features import build_features


In [17]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")

In [22]:
resultados = {}
for columna in COLS_A_PREPROCESAR_TEXTO:
    valores_unicos = df_procesada[columna].dropna().unique().astype(str)
    vectorizador, matriz_sparse = build_features.crear_matriz_sparse_para_texto(valores_unicos)

    cantidad_de_grupos = int(round(valores_unicos.shape[0] / 2, 0))

    modelo = KMeans(n_clusters=cantidad_de_grupos, n_init="auto", random_state=1)
    modelo.fit(matriz_sparse)
    clasificacion = clasificacion_diagnosticos = pd.DataFrame(
        {columna: valores_unicos, "cluster": modelo.labels_}
    ).sort_values("cluster")

    resultados[columna] = clasificacion

In [23]:
with pd.ExcelWriter("../data/processed/agrupacion_columnas.xlsx") as file:
    for nombre_col, df_agrupada in resultados.items():
        df_agrupada.to_excel(file, sheet_name=nombre_col, index=False)